### Imports

In [2]:
import os
import cv2
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import List
from collections import namedtuple
from torchvision.transforms import Compose
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

### Labels

In [3]:
#--------------------------------------------------------------------------------
# Definitions
#--------------------------------------------------------------------------------

# a label and all meta information
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).

    'trainId'     , # An integer ID that overwrites the ID above, when creating ground truth
                    # images for training.
                    # For training, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )


#--------------------------------------------------------------------------------
# A list of all labels
#--------------------------------------------------------------------------------

# Please adapt the train IDs as appropriate for you approach.
# Note that you might want to ignore labels with ID 255 during training.
# Make sure to provide your results using the original IDs and not the training IDs.
# Note that many IDs are ignored in evaluation and thus you never need to predict these!

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'ground'          , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'ground'          , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'ground'          , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'ground'          , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , 34 ,       19 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

# function to get the label from the name
def find_label(name):
    for label in labels:
        if label.name == name:
            return label.trainId, label.color
    return 255, (0,0,0)

### Loading Data

In [4]:
#For Working Within Kaggle - Seif
#--------------------------------------------------------------------------------
# train_images_folder = '/kaggle/input/cityscapes-segmentation/images/train'
# train_labels_folder = '/kaggle/input/cityscapes-segmentation/labels/train'

# training_csv_file_path = '/kaggle/working/cityscapes_training.csv'
# val_csv_file_path = '/kaggle/working/cityscapes_val.csv'
#--------------------------------------------------------------------------------

#For Working Locally - Youssif
#--------------------------------------------------------------------------------
current_dir = os.path.dirname(os.getcwd())
train_images_folder = os.path.join(current_dir, 'DataSet\\Images\\Train_Set')
train_labels_folder = os.path.join(current_dir, 'DataSet\\Labels\\train')

training_csv_file_path = os.path.join(current_dir, 'DataSet\\cityscapes_training.csv')
val_csv_file_path = os.path.join(current_dir, 'DataSet\\cityscapes_val.csv')
#--------------------------------------------------------------------------------

data = {
    'image_path': [],
    'label_mask_path': [],
    'label_json_path': []
}

for city in os.listdir(train_images_folder):
    city_image_folder = os.path.join(train_images_folder, city)
    city_label_folder = os.path.join(train_labels_folder, city)
    # print(f"Processing {city} ...")
    
    if os.path.isdir(city_image_folder):
        for image_file in os.listdir(city_image_folder):
            # print(f"Processing {image_file} ...")
            if image_file.endswith('_leftImg8bit.png'):
                image_name_base = image_file.replace('_leftImg8bit.png', '')
                label_mask_file = f"{image_name_base}_gtFine_labelTrainIds.png"
                label_json_file = f"{image_name_base}_gtFine_polygons.json"
                
                data['image_path'].append(os.path.join(city_image_folder, image_file))
                data['label_mask_path'].append(os.path.join(city_label_folder, label_mask_file))
                data['label_json_path'].append(os.path.join(city_label_folder, label_json_file))

df = pd.DataFrame(data)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=23, shuffle=True)

train_df.to_csv(training_csv_file_path, index=False)
val_df.to_csv(val_csv_file_path, index=False)

In [5]:
#For Working Within Kaggle - Seif
#--------------------------------------------------------------------------------
# test_images_folder = '/kaggle/input/cityscapes-segmentation/images/test'

# test_csv_file_path = '/kaggle/working/cityscapes_test.csv'
#--------------------------------------------------------------------------------

#For Working Locally - Youssif
#--------------------------------------------------------------------------------
test_images_folder = os.path.join(current_dir, 'DataSet\\Images\\Test_Set')

test_csv_file_path = os.path.join(current_dir, 'DataSet\\cityscapes_test.csv')
#--------------------------------------------------------------------------------

test_data = {
    'image_path': []
}

for city in os.listdir(test_images_folder):
    city_image_folder = os.path.join(test_images_folder, city)
    
    if os.path.isdir(city_image_folder):
        for image_file in os.listdir(city_image_folder):
            if image_file.endswith('_leftImg8bit.png'):
                test_data['image_path'].append(os.path.join(city_image_folder, image_file))

test_df = pd.DataFrame(test_data)

test_df.to_csv(test_csv_file_path, index=False)

### EDA

In [ ]:
# Displaying A Sample of Images and Masks
def show_img_mask(row):
    image = cv2.cvtColor(cv2.imread(row['image_path']), cv2.COLOR_BGR2RGB)
    mask = cv2.imread(row['label_mask_path'])

    json_data = json.load(open(row['label_json_path']))

    for obj in json_data['objects']:
        label = obj['label']
        trainId, color = find_label(label)
        if trainId != 255:
            mask = cv2.fillPoly(mask, np.array([obj['polygon']], dtype=np.int32), color)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[0].axis('off')
    ax[0].set_title('Image')
    ax[1].imshow(mask, cmap='gray')
    ax[1].axis('off')
    ax[1].set_title('Mask')
    plt.show()

rSample = train_df.sample(5)
for index, row in rSample.iterrows():
    show_img_mask(row)

In [ ]:
# Analyzing the Distribution of the Labels accross the train Dataset
labels_freq = {label.name: 0 for label in labels}
labels_not_found = {}

dimensions_freq = {}

for index, row in train_df.iterrows():
    json_data = json.load(open(row['label_json_path']))
    
    dimensions = f"{json_data['imgWidth']}x{json_data['imgHeight']}"
    dimensions_freq[dimensions] = dimensions_freq.get(dimensions, 0) + 1

    for obj in json_data['objects']:
        label = obj['label']
        if label in labels_freq:
            labels_freq[label] += 1
        else:
            if label not in labels_not_found:
                labels_not_found[label] = 1
            else:
                labels_not_found[label] += 1
        
plt.figure(figsize=(20, 10))
plt.bar(labels_freq.keys(), labels_freq.values())
plt.xticks(rotation=90)
plt.show()
print(labels_not_found)

In [ ]:
# Bar Plot for the Dimensions of the Images
plt.figure(figsize=(20, 10))
plt.bar(dimensions_freq.keys(), dimensions_freq.values())
plt.xticks(rotation=90)
plt.show()

# All Images have the same dimensions

In [ ]:
# Working...

### PreProcessing & The rest

In [8]:
# Class of Loading Data into Images and Masks
class LoadData():
    def __init__(self, keys):
        self.keys = keys
        self.key_size = len(keys)

    def __call__(self, sample):
        if 'image_path' in self.keys and sample.get('image_path', None) is not None:
            image = cv2.imread(sample['image_path']) 
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
        else:
            image = None

        if self.key_size == 3:
            mask_path = sample.get("label_mask_path", None)
            if mask_path:
                mask = cv2.imread(mask_path)  
            else:
                mask = None
            
            json_path = sample.get("label_json_path", None)
            if json_path:
                with open(json_path, 'r') as f:
                    json_data = json.load(f)

                img_height = json_data["imgHeight"]
                img_width = json_data["imgWidth"]
                objects = json_data["objects"]

                for obj in objects:
                    label = obj['label']
                    polygon = np.array(obj['polygon'], dtype=np.int32)
                    id_, color = find_label(label) 
                    # Skip ignored labels (trainId 255)
                    if id_ != 255:
                        cv2.fillPoly(mask, [polygon], color=color)
            
            return (image, mask)

        return (image, None)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [10]:
class ConvToTensor():
    def __call__(self, sample):
        # Convert the image to a tensor and permute dimensions (HWC to CHW)
        image_tensor = torch.tensor(sample[0].transpose(2, 0, 1), dtype=torch.float32, device=device) / 255.0 # normalize
        if sample[1] is not None:
            mask_tensor = torch.tensor(sample[1], dtype=torch.long, device=device)
        else:
            mask_tensor = None
        return (image_tensor, mask_tensor)

In [11]:
#TODO : TRAIN DATA PREPROCESSING PIPELINE
csv_keys = ['image_path', 'label_mask_path', 'label_json_path']

Train_data_transform=Compose([
    LoadData(csv_keys),  
    ConvToTensor()
])


Valid_data_transform=Compose([
   LoadData(csv_keys),
    ConvToTensor()
])


Test_data_transform=Compose([
   LoadData(['image_path']),
   ConvToTensor()
])

In [12]:
class Dataset(Dataset):
    def __init__(self, input_dataframe: pd.DataFrame, KeysOfInterest: List[str], data_transform:Compose): # TODO: add parameters if needed
        self.koi = KeysOfInterest
        self.input_dataframe = input_dataframe[self.koi]
        self.data_transform=data_transform

    def __getitem__(self, item: int):
        sample = self.input_dataframe.iloc[item].to_dict()
        transformed_sample = self.data_transform(sample)
        return transformed_sample
        
    def __len__(self):
        return len(self.input_dataframe)

In [13]:
#TODO : Initilize your datasets

ds_train=Dataset(input_dataframe=train_df,
                KeysOfInterest=csv_keys,
                data_transform=Train_data_transform)

ds_val=Dataset(input_dataframe=val_df,
                KeysOfInterest=csv_keys,
                data_transform=Valid_data_transform)

ds_test=Dataset(input_dataframe=test_df,
                KeysOfInterest=["image_path"],
                data_transform=Test_data_transform)


In [14]:
BATCH_SIZE = 16

dl_train = DataLoader(dataset=ds_train,batch_size= BATCH_SIZE,shuffle=True)
dl_val = DataLoader(dataset=ds_val,batch_size= BATCH_SIZE ,num_workers=4 ,prefetch_factor=8,shuffle=True)
dl_test = DataLoader(dataset=ds_test,batch_size= 1 ,num_workers=4 ,prefetch_factor=4,shuffle=False)

In [ ]:
#TODO : Show samples from your data loaders

def show_sample(image_tensor, mask_tensor=None):

    if image_tensor.is_cuda:
        image_tensor = image_tensor.cpu()
    if mask_tensor is not None and mask_tensor.is_cuda:
        mask_tensor = mask_tensor.cpu()
    
    # Convert the image tensor back to HWC format for plotting (CHW -> HWC)
    image = image_tensor.permute(1, 2, 0).numpy()  # From CHW to HWC
    image = (image * 255).astype("uint8")  # Scale back to [0, 255] for visualization

    plt.figure(figsize=(10, 5))
    
    # Plot the image
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Image")
    plt.axis('off')

    # Plot the mask (if available)
    if mask_tensor is not None:
        plt.subplot(1, 2, 2)
        plt.imshow(mask_tensor.numpy(), cmap='gray')
        plt.title("Mask")
        plt.axis('off')

    plt.show()

# Iterate through the dataloader and show a few samples
def show_data_samples(dataloader, num_samples=5):
    for i, (image_tensor, mask_tensor) in enumerate(dataloader):
        if i >= num_samples:
            break
        print(f"Sample {i+1}:")
        show_sample(image_tensor[0], mask_tensor[0] if mask_tensor is not None else None)

# Example usage to show samples from the training dataloader
show_data_samples(dl_train, num_samples=5)

In [16]:
#TODO : Write the model you are going to use (Pytorch)

In [17]:
#TODO : Write the loss function you are going to use

In [18]:
#TODO : Write the evaluation metrics you are going to use

In [19]:
#TODO : Write your own Training loop using pytorch

In [20]:
#TODO : Plot losses and metrics graphs

In [21]:
#TODO : Test your model and show some samples